In [2]:
import pandas as pd
from sqlalchemy import Inspector
from sqlalchemy.dialects.mssql import NVARCHAR, FLOAT, DATETIME2, INTEGER
from sqlalchemy.engine import Engine
from aom.tools.sql_connect import SQLConnector
from aom.definitions import DATA_DIR
conn = SQLConnector(database='mssql', db_profile='AzureSQL')
conn.inspector()

ModuleNotFoundError: No module named 'pandas'

In [3]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Device 0:", torch.cuda.get_device_name(0))
    x = torch.randn(3,3, device="cuda"); print("OK on", x.device)

2.8.0+cpu
CUDA available: False
GPU count: 0


In [17]:
def read_table(conn_eng: Engine, table_name:str) -> pd.DataFrame:
    query = f"""
            SELECT * FROM {table_name}
            """
    df = pd.read_sql(query, conn_eng)
    return df

In [ ]:
def create_table(conn_eng: Engine, table_name:str, ddl_create: str = None, replace: bool = False) -> None:
    """Create dbo.asset_data with required schema. Set replace=True to DROP+CREATE."""
    ddl_drop = f"IF OBJECT_ID('dbo.{table_name}','U') IS NOT NULL DROP TABLE dbo.{table_name};"
    ddl_create = f"""
    IF OBJECT_ID('dbo.{table_name}','U') IS NULL
    CREATE TABLE dbo.{table_name} (
        timestamp    DATETIME2    NOT NULL,
        asset_id     NVARCHAR(64) NOT NULL,
        temperature_c  FLOAT        NULL,
        vibration_mm_s    FLOAT        NULL,
        anomaly      INT          NULL
    );
    """
    with conn_eng.begin() as connection:
        if replace:
            connection.exec_driver_sql(ddl_drop)
        connection.exec_driver_sql(ddl_create)

In [19]:
def insert_data(conn_eng: Engine, df, table_name:str, sch_name:str,
                dtype: dict, insp, append: bool = False) -> None:
    if append:
        flg = "append"
    else:
        flg = "replace"

    if "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"], format="%m/%d/%Y %H:%M", errors="raise")
        # strip timezone if any
        if getattr(df["timestamp"].dt, "tz", None) is not None:
            df["timestamp"] = df["timestamp"].dt.tz_convert("UTC").dt.tz_localize(None)

    exists = insp.has_table(table_name, schema=sch_name)

    if not exists:
        df.head(0).to_sql(name=table_name, con=conn_eng, schema=sch_name,
                          if_exists="fail", index=False, dtype=dtype)

    df.to_sql(name=table_name,
              con=conn_eng,
              schema=sch_name,
              if_exists=flg,
              index=False,
              dtype=dtype)

In [20]:
sensor = pd.read_csv(f'{DATA_DIR}/test_ts_data.csv')
sensor = sensor[['timestamp', 'asset_id', 'temperature_c', 'vibration_mm_s']].copy()
meta = pd.read_csv(f'{DATA_DIR}/test_meta.csv')

In [24]:
# dtype_map = {
#     "timestamp": DATETIME2,
#     "asset_id": NVARCHAR(length=64),
#     "temperature_c": FLOAT,
#     "vibration_mm_s": FLOAT,
#     # "anomaly": INTEGER,
# }
dtype_map = {
    "asset_id": NVARCHAR(length=64),
    "maker": NVARCHAR(length=64),
    "rated_power_kw": FLOAT,
    "rated_voltage_v": FLOAT,
}

In [26]:
conn = SQLConnector(database='mssql', db_profile='AzureSQL')
conn_engine = conn.get_engine()
conn_insp = conn.inspector()
insert_data(conn_engine, table_name='asset_meta', sch_name='dbo',
            dtype=dtype_map, df=meta, append=False, insp=conn_insp)

2025-09-26 09:30:25+0000 | INFO | SQLConnector | Database profile: 'AzureSQL'


schema ['db_accessadmin', 'db_backupoperator', 'db_datareader', 'db_datawriter', 'db_ddladmin', 'db_denydatareader', 'db_denydatawriter', 'db_owner', 'db_securityadmin', 'dbo', 'guest', 'INFORMATION_SCHEMA', 'sys']
tables ['asset_data', 'asset_meta', 'DemoSTPPlantMapping', 'DemoWorkManagementPump']


In [27]:
conn = SQLConnector(database='mssql',db_profile='AzureSQL')
conn.inspector()
engine = conn.get_engine()
# create_table(engine, table_name='asset_data', replace=True)
check_db = read_table(engine, 'asset_meta')
check_db

2025-09-26 09:30:27+0000 | INFO | SQLConnector | Database profile: 'AzureSQL'


schema ['db_accessadmin', 'db_backupoperator', 'db_datareader', 'db_datawriter', 'db_ddladmin', 'db_denydatareader', 'db_denydatawriter', 'db_owner', 'db_securityadmin', 'dbo', 'guest', 'INFORMATION_SCHEMA', 'sys']
tables ['asset_data', 'asset_meta', 'DemoSTPPlantMapping', 'DemoWorkManagementPump']


,asset_id,maker,rated_power_kw,rated_voltage_v
0,motor_1,ABB,75.0,415.0
1,motor_2,Siemens,560.0,690.0
2,motor_3,ABB,55.0,415.0
